In [45]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy.ndimage as nd
import scipy

from skimage.feature import hog
from skimage import data, color, exposure
from sklearn.cluster import KMeans

import random

In [51]:
vocabulary = np.load('/oasis/projects/nsf/csd395/ruogu/vocabulary/vocabulary.npy');
M = vocabulary.shape[0];
L = 3;
dataDir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/patches/';
svmDataDir = '/oasis/projects/nsf/csd395/ruogu/svm/';
classes = ['12N', '5N', '7n', '7N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 'Bg'];

In [52]:
kmeans = KMeans(init='random', n_clusters=M, n_init=10);
kmeans.fit(vocabulary);

In [118]:
# Return the histogram of an image by applying SIFT + K-means + SPM
# image: the input grayscaled image
# kmeans: python kmeans object used to predict
#             feature type of a new sift descriptor
# M: number of channels (feature typs)
# L: number of SPM levels.
def getHistogram(image, kmeans, M, L):
    x_size = image.shape[0];
    y_size = image.shape[1];
    # Get SIFT key points and descriptor.
    sift = cv2.SIFT();
    key_point, descriptor = sift.detectAndCompute(image, None);
    # Map key points with type.
    if descriptor is None:
        return None;
    cluster_labels = kmeans.predict(descriptor);
    cluster_labels = cluster_labels + 1; # so label starts with 1
    # Generate position list of key_points
    positions = [];
    for k in key_point:
        positions.append((int(k.pt[0]), int(k.pt[1])));
    label_map = np.zeros(shape=(x_size, y_size), dtype=np.int64);
    positions = np.asarray(positions);
    label_map[positions[:,1], positions[:,0]] = cluster_labels;
    # Level 0
    weight_0 = 1.0 / (2**L);
    histogram = np.bincount(label_map.flatten(), minlength=(M + 1))[1:];
    histogram = histogram * weight_0;
    # Other levels
    for l in range(1, L):
        grid_x = x_size / (2**l);
        grid_y = y_size / (2**l);
        weight = 1.0 / (2**(L - l + 1));
        for i in range(0, (2**l)):
            x_min = i * grid_x;
            x_max = x_min + grid_x;
            for j in range(0, (2**l)):
                y_min = j * grid_y;
                y_max = y_min + grid_y;
                # Count and update histogram from this grid.
                histogram_tmp = np.bincount(label_map[x_min:x_max, y_min:y_max].flatten(), minlength=(M+1))[1:]
                histogram_tmp = histogram_tmp * weight;
                histogram = np.hstack((histogram, histogram_tmp));
    return (histogram / len(key_point)); # Normalization

In [ ]:
#from time import time
fileNames = os.listdir(dataDir);
X = [];
y = [];
for fileName in fileNames:
    print fileName;
    images = np.load(dataDir + fileName);
    label = -1;
    # Get class label
    if fileName[7] == '_':
        className = fileName.partition('_')[-1].rpartition('_')[0];
        label = classes.index(className);
    else: # background
        label = 9;
    print label;
    
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
        # t0 = time();
        h = getHistogram(img, kmeans, M, L);
        # t1 = time();
        # print (t1-t0);
        if h is None:
            continue;
        else:
            X.append(h);
            y.append(label);